# GAN-FS

## 1. 导入包

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## 2. 定义预处理函数

In [2]:
def convertstringtonumber(df, lst):
    """ 字符串转为数字型"""
    for n in range(len(lst)):
        df = df.replace(lst[n], n)
    return df

def scalex(X):
    """ 数值标准化"""
    nmin, nmax = 0.0, 1.0
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (nmax - nmin) + nmin
    return X_scaled

## 3. 读取数据集

In [3]:
col_names = ["Duration", "Protocol_type", "Service", "Flag", "Src_bytes", "Dst_bytes", 
       "Land", "Wrong_fragment", "Urgent", "Hot", "Num_failed_logins", "Logged_in", 
       "Num_compromised", "Root_shell", "Su_attempted", "Num_root", "Num_file_creations",
       "Num_shells", "Num_access_files", "Num_outbound_cmds", "Is_hot_login", 
       "Is_guest_login", "Count", "Srv_count", "Serror_rate", "Srv_serror_rate",
       "Rerror_rate", "Srv_rerror_rate", "Same_srv_rate", "Diff_srv_rate",
       "Srv_diff_host_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_srv_rate", 
       "Dst_host_diff_srv_rate", "Dst_host_same_src_port_rate", "Dst_host_srv_diff_host_rate",
       "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "Dst_host_rerror_rate",
       "Dst_host_srv_rerror_rate", "attack_type", "Class"]

df_train = pd.read_csv("./dataset/KDDTrain+.txt", sep=",", header=None)
df_test = pd.read_csv("./dataset/KDDTest+.txt", sep=",", header=None)

df_train.columns = col_names
df_test.columns = col_names

df_train = df_train.drop(columns=['Class'])
df_test = df_test.drop(columns=['Class'])

print('训练集维度:',df_train.shape)
print('测试集维度:',df_test.shape)

训练集维度: (125973, 42)
测试集维度: (22544, 42)


## 4. 数据预处理

In [4]:
protocol_type = ['icmp', 'tcp', 'udp']
service = ['IRC','X11','Z39_50','aol','auth','bgp','courier','csnet_ns','ctf',
           'daytime','discard','domain','domain_u','echo','eco_i','ecr_i','efs',
           'exec','finger','ftp','ftp_data','gopher','harvest','hostnames','http',
           'http_2784','http_443','http_8001','imap4','iso_tsap','klogin','kshell',
           'ldap','link','login','mtp','name','netbios_dgm','netbios_ns',
           'netbios_ssn','netstat','nnsp','nntp','ntp_u','other','pm_dump','pop_2',
           'pop_3','printer','private','red_i','remote_job','rje','shell','smtp',
           'sql_net','ssh','sunrpc','supdup','systat','telnet','tftp_u','tim_i',
           'time','urh_i','urp_i','uucp','uucp_path','vmnet','whois']
flag = ['OTH','REJ','RSTO','RSTOS0','RSTR','S0','S1','S2','S3','SF','SH']

df_train = convertstringtonumber(df_train, protocol_type)
df_train = convertstringtonumber(df_train, service)
df_train = convertstringtonumber(df_train, flag)

df_test = convertstringtonumber(df_test, protocol_type)
df_test = convertstringtonumber(df_test, service)
df_test = convertstringtonumber(df_test, flag)
        
for n in range(len(col_names)-2): #df_train标准化
        m = col_names[n]
        if (np.max(df_train[m]) > 1): 
            if (len(np.unique(df_train[m])) > 1):
                df_train[m] = scalex(df_train[m])
            else:
                df_train[m] = np.int64(1)
                
for n in range(len(col_names)-2): #df_train标准化
        m = col_names[n]
        if (np.max(df_test[m]) > 1): 
            if (len(np.unique(df_test[m])) > 1):
                df_test[m] = scalex(df_test[m])
            else:
                df_test[m] = np.int64(1)

df_test.head(5)

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,attack_type
0,0.000000,0.5,0.710145,0.1,0.000000e+00,0.000000,0,0.0,0.0,0.0,...,0.039216,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune
1,0.000000,0.5,0.710145,0.1,0.000000e+00,0.000000,0,0.0,0.0,0.0,...,0.003922,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune
2,0.000035,0.5,0.289855,0.9,2.066513e-04,0.000000,0,0.0,0.0,0.0,...,0.337255,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal
3,0.000000,0.0,0.202899,0.9,3.183413e-07,0.000000,0,0.0,0.0,0.0,...,0.223529,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint
4,0.000017,0.5,0.869565,0.2,0.000000e+00,0.000011,0,0.0,0.0,0.0,...,0.337255,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan


In [5]:
labeldf_train = df_train['attack_type']
labeldf_test = df_test['attack_type']

# 用数字代替攻击类型
newlabeldf_train = labeldf_train.replace({ 'normal' : 0,'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                            'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1,
                            'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                            'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1})

newlabeldf_test = labeldf_test.replace({ 'normal' : 0,'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                            'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1,
                            'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                            'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1})

df_train['attack_type'] = newlabeldf_train
df_test['attack_type'] = newlabeldf_test

df_test.head(5)

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,attack_type
0,0.000000,0.5,0.710145,0.1,0.000000e+00,0.000000,0,0.0,0.0,0.0,...,0.039216,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1
1,0.000000,0.5,0.710145,0.1,0.000000e+00,0.000000,0,0.0,0.0,0.0,...,0.003922,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1
2,0.000035,0.5,0.289855,0.9,2.066513e-04,0.000000,0,0.0,0.0,0.0,...,0.337255,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,0
3,0.000000,0.0,0.202899,0.9,3.183413e-07,0.000000,0,0.0,0.0,0.0,...,0.223529,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,1
4,0.000017,0.5,0.869565,0.2,0.000000e+00,0.000011,0,0.0,0.0,0.0,...,0.337255,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,1


## 5. 过采样

In [6]:
# 读取过采样样本

df_R2L = pd.read_csv("./generate_sample/generated_R2L.csv",header=None, names = col_names[0:-2])
df_U2R = pd.read_csv("./generate_sample/generated_U2R.csv",header=None, names = col_names[0:-2])

df_R2L["attack_type"] = 1
df_U2R["attack_type"] = 1

In [7]:
# 将过采样样本加入训练集

print("过采样前训练集样本数量：",df_train.shape)
df_train = df_train.append(df_R2L)
df_train = df_train.append(df_U2R)
print("过采样后训练集样本数量：",df_train.shape)

df_train.index = range(len(df_train))

过采样前训练集样本数量： (125973, 42)
过采样后训练集样本数量： (145973, 42)


In [8]:
df_train.head(5)

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate,attack_type
0,0.0,0.5,0.289855,0.9,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.0,...,0.098039,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0
1,0.0,1.0,0.637681,0.9,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,...,0.003922,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
2,0.0,0.5,0.710145,0.5,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.101961,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
3,0.0,0.5,0.347826,0.9,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,...,1.000000,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
4,0.0,0.5,0.347826,0.9,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,...,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0


## 6. 拆分数据集

In [9]:
# 二分类，分类标签为最后一列，多分类标签后边再使用
train_X = df_train.iloc[:,0:-1]
train_y = df_train.iloc[:,-1]

test_X = df_test.iloc[:,0:-1]
test_y = df_test.iloc[:,-1]

In [10]:
train_X.head(5)

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,...,Dst_host_count,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,Dst_host_srv_rerror_rate
0,0.0,0.5,0.289855,0.9,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.0,...,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0.0,1.0,0.637681,0.9,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,...,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0.0,0.5,0.710145,0.5,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,1.000000,0.101961,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0.0,0.5,0.347826,0.9,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,...,0.117647,1.000000,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0.0,0.5,0.347826,0.9,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [11]:
train_y.head(5)

0    0
1    0
2    1
3    0
4    0
Name: attack_type, dtype: int64

## 7. 特征选择

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_classif
np.seterr(divide='ignore', invalid='ignore')
selector = SelectPercentile(f_classif, percentile=45)
fs_X = selector.fit_transform(train_X,train_y) # fs_X是特征选择后的数据

In [ ]:
true = selector.get_support()

newcolindex=[i for i, x in enumerate(true) if x]
print("所选特征索引：",newcolindex)

newcolname=list( col_names[i] for i in newcolindex )
print("所选特征名：",newcolname)

In [ ]:
feature_col = newcolname
train_fs_X = train_X[feature_col]
test_fs_X = test_X[feature_col]

In [ ]:
train_fs_X.head(5)

In [ ]:
test_fs_X.head(5)

## 7. ml实验

### 导入模型

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [20]:
DT_b = DecisionTreeClassifier(random_state=0) # 决策树
RF_b = RandomForestClassifier(n_estimators=10, random_state=0) # 随机森林
GB_b = GradientBoostingClassifier(loss='deviance', learning_rate=0.5, n_estimators=75, max_features='auto',random_state=0) # 梯度提升树
SVM_b = svm.SVC() # SVM
NB_b = BernoulliNB() # 朴素贝叶斯
KNN_b = KNeighborsClassifier(n_neighbors=3, n_jobs=1)
MLP_b = MLPClassifier(solver='sgd',hidden_layer_sizes=(5, 2), max_iter=10000, random_state=1) # 多层感知机

### 6.1 决策树

In [ ]:
DT_b.fit(train_fs_X,train_y)
DT_pred_y = DT_b.predict(test_fs_X)
DT_t = classification_report(test_y,DT_pred_y,digits=4)
print("______________________________________________________________") 
print("DT分类结果：")
print(DT_t)
print("______________________________________________________________")

### 6.2 随机森林

In [ ]:
RF_b.fit(train_fs_X,train_y)
RF_pred_y = RF_b.predict(test_fs_X)
RF_t = classification_report(test_y,RF_pred_y,digits=4)
print("______________________________________________________________")
print("RF分类结果：")
print(RF_t)
print("______________________________________________________________")

### 6.3 梯度提升树

In [ ]:
GB_b.fit(train_fs_X,train_y)
GB_pred_y = GB_b.predict(test_fs_X)
GB_t = classification_report(test_y,GB_pred_y,digits=4)
print("______________________________________________________________") 
print("GBDT分类结果：")
print(GB_t)
print("______________________________________________________________")

### 6.4 支持向量机

In [ ]:
SVM_b.fit(train_fs_X,train_y)
SVM_b.score(train_fs_X,train_y)
SVM_pred_y = SVM_b.predict(test_fs_X)
SVM_t = classification_report(test_y,SVM_pred_y,digits=4)
print("______________________________________________________________") 
print("SVM分类结果：")
print(SVM_t)
print("______________________________________________________________") 

### 6.5 朴素贝叶斯

In [ ]:
NB_b.fit(train_fs_X,train_y)
NB_pred_y = NB_b.predict(test_fs_X)
NB_t = classification_report(test_y,NB_pred_y,digits=4)
print("______________________________________________________________") 
print("NB分类结果：")
print(NB_t)
print("______________________________________________________________")

### 6.6 K-NN

In [ ]:
KNN_b.fit(train_fs_X,train_y)
KNN_pred_y = KNN_b.predict(test_fs_X)
KNN_t = classification_report(test_y,KNN_pred_y,digits=4)
print("______________________________________________________________") 
print("K-NN分类结果：")
print(KNN_t)
print("______________________________________________________________")

### 6.7 多层感知机（MLP）

In [ ]:
MLP_b.fit(train_fs_X,train_y)
MLP_pred_y = MLP_b.predict(test_fs_X)
MLP_t = classification_report(test_y,MLP_pred_y,digits=4)
print("______________________________________________________________") 
print("MLP分类结果：")
print(MLP_t)
print("______________________________________________________________")